In [ ]:
import os

from tqdm import tqdm
from json import dumps
from ujson import load as json_load

import matplotlib as plt
import numpy as np
import random
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched

from data_loader import fetch_dataloader
from CNN import BinaryClimbCNN

from PIL import Image

In [ ]:
if __name__ == "__main__":

    data_loaders, n_examples = fetch_dataloader(['train', 'val', 'test'], 0)
    batch_size = 64

    train_dataloader = data_loaders['train']
    val_dataloader = data_loaders['val']
    test_dataloader = data_loaders['test']

    GRID_DIMS = (18, 11) # dimensions

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    classes = ['6A+', '6B', '6B+', '6C', '6C+', '7A', '7A+', '7B', '7B+', '7C', '7C+', '8A', '8A+', '8B', '8B+']
    n_classes = len(classes)

    model = BinaryClimbCNN(n_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

    def train(dataloader, model, loss_fn, optimizer):
        model.train()
        loss_list, accuracy_list = [], []
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            X = X.float()

            # Compute prediction and loss
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            correct = (pred.argmax(1) == y).type(torch.float).sum().item()
            accuracy = correct / X.shape[0]

            loss_list.append(loss.item())
            accuracy_list.append(accuracy)

            if batch % 100 == 0:
                print(f"loss: {loss.item():>7f}, accuracy: {accuracy * 100:>0.1f}%")

        return loss_list, accuracy_list


    def test(dataloader, model):
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                X = X.float()
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= len(dataloader)
        accuracy = correct / len(dataloader.dataset)
        return test_loss, accuracy
    
    epochs = 1
    train_loss, train_accuracy, test_loss, test_accuracy = [], [], [], []

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}\n-------------------------------")
        epoch_train_loss, epoch_train_accuracy = train(train_dataloader, model, loss_fn, optimizer)
        epoch_test_loss, epoch_test_accuracy = test(test_dataloader, model)

        train_loss.append(np.mean(epoch_train_loss))
        train_accuracy.append(np.mean(epoch_train_accuracy))
        test_loss.append(epoch_test_loss)
        test_accuracy.append(epoch_test_accuracy)



(16025, 18, 11) (16025,) 16025
(2009, 18, 11) (2009,) 2009
(2009, 18, 11) (2009,) 2009
Epoch 1
-------------------------------
loss: 2.916102, accuracy: 3.1%
loss: 2.029990, accuracy: 31.2%
loss: 1.885072, accuracy: 29.7%


In [7]:
print(train_loss)

[2.0686424315213205]
